<a href="https://colab.research.google.com/github/yeaeunJi/deep_learning-/blob/main/%EC%96%B4%ED%85%90%EC%85%98(Attention).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 어텐션의 구조
- 어텐션 메커니즘을 통해 seq2seq는 인간과 같이 필요한 정보에만 주목할 수 있게되었으며, 지금까지 seq2seq에 있던 문제도 해결할 수 있게 됨

### seq2seq의 문제점
- seq2seq에서는 Encoder의 출력이 입력 데이터의 길이와는 관계없이 항상 일정한 고정 길이 벡터였음
  - 즉, 필요한 정보가 해당 벡터에 다 담기지 못하게 되는 경우가 발생

### Encoder 개선
- Encoder 입력 데이터의 길이에 따라 출력의 길이가 바뀌도록 개선
  - 시각 별 LSTM 계층의 은닉 상태 벡터를 모두 이용하면 입력 데이터와 길이가 같은 수의 벡터를 얻을 수 있음
  - Encoder의 출력 hs는 각 단어에 해당하는 벡터들의 집합
  

In [5]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/deep_learning2/

Mounted at /content/drive
/content/drive/MyDrive/deep_learning2


### Decoder 개선1
- 입력과 출력의 여러 단어 중 어떤 단어끼리 대응되는가를 seq2seq에 학습
- 필요한 정보에만 주목하여 그 정보로부터 시계열 변환을 수행하는 것(이러한 구조를 어텐션이라고 부름) 
- 각 단어의 중요도를 나타내는 가중치 a로 가중치합을 이용해 맥락 벡터를 얻을 수 있음

In [6]:
## 맥락 벡터를 구하는 Weight Sum 계층
class WeightSum :
  def __init__(self) :
    self.params, self.grads = [], []
    self.cache = None

  def forward(self, hs, a) :
    N, T, H = hs.shape

    ar = a.reshape(N, T , 1).repeat(H, axis=2)
    t = hs * ar
    c = np.sum(t, axis=1)

    self.cache = (hs, ar)
    return c
  
  def backward(self, dc) :
    hs, ar = self.cache
    N, T, H = hs.shape

    dt = dc.reshape(N, 1, H).repeat(T, axis=1) # sum의 역전파
    dar = dt * hs
    dhs = dt * ar
    da = np.sum(dar, axis=2) # repeat의 역전파
    return dhs, da

### Decoder 개선2
- 각 방법의 가중치 a를 구하는 방법
  - 데이터로부터 자동으로 학습할 수 있도록 준비 필요
          

In [7]:
from common.np import * 
from common.layers import Softmax

class AttentionWeight :
  def __init__(self) :
    self.params, self.grads = [], []
    self.softmax = Softmax()
    self.cache = None
  
  def forward(self, hs, h) :
    N, T, H = hs.shape

    hr = h.reshape(N, 1, H).repeat(T, axis=1)
    t = hs * hr
    s = np.sum(t, axis=2)
    a = self.softmax.forward(s)

    self.cache = (hs, hr)
    return a

  def backward(self, da) :
    hs, hr = self.cache
    N, T, H = hs.shape

    ds = self.softmax.backward(da)
    dt = ds.reshape(N, T, 1).repeat(H, axis=2)
    dhs = dt * hr
    dhr = dt * hs
    dh = np.sum(dhr, axis=1)

    return dhs, dh

### Decoder 개선3
- Atteion Weight 계층
  - Encoder가 출력하는 각 단어의 벡터  hs에 주목하여 해당 단어의 가중치 a를 구함

- Weight Sum 계층
  - a와 hs의 가중치 합을 구하고 그 결과를 맥락 벡터 c로 출력

- Encoder가 건네주는 정보 hs에서 중요한 원소에 주목하여, 이를 바탕으로 맥락 벡터를 구하여 위쪽 계층으로 전파

In [8]:
class Attention :
  def __init__(self) :
    self.params, self.grads = [], []
    self.attention_weight_layer = AttentionWeight()
    self.weight_sum_layer = WeightSum()
    self.attention_weight = None

  def forward(self, hs, h) :
    a = self.attention_weight_layer.forward(hs, h)
    out = self.weight_sum_layer.forward(hs, a)
    self.attention_weight = a
    return out

  def backward(self, dout) :
    dhs0, da = self.weight_sum_layer.backward(dout)
    dhs1, dh = self.attention_weight_layer.backward(da)
    dhs = dhs0 + dhs1
    return dhs, dh

In [9]:
class TimeAttention :
  def __init__(self) :
    self.params, self.grads = [], []
    self.layers = None
    self.attention_weights = None
  
  def forward(self, hs_enc, hs_dec) :
    N, T, H = hs_dec.shape
    out = np.empty_like(hs_dec)
    self.layers = []
    self.attention_weights = []

    for t in range(T) :
      layer = Attention()
      out[:, t, :] = layer.forward(hs_enc, hs_dec[:, t, :])
      self.layers.append(layer)
      self.attention_weights.append(layer.attention_weight)

    return out

  def backward(self, dout) :
    N, T, H = dout.shape
    dhs_enc = 0
    dhs_dec = np.empty_like(dout)

    for t in range(T) :
      layer = self.layers[t]
      dhs, dh = layer.backward(dout[: ,t, :])
      dhs_enc += dhs
      dhs_dec[:, t, :] = dh

    return dhs_enc, dhs_dec 

## 어텐션을 갖춘 seq2seq 구현

### Encoder 구현
- 모든 은닉 상태를 반환하도록 forward() 메서드가 구현됨

In [10]:
from common.time_layers import *
from ch07.seq2seq import Encoder, Seq2seq
from ch08.attention_layer import TimeAttention

class AttentionEncdoer(Encoder) :
  def forward(self,xs):
    xs = self.embed.forward(xs)
    hs = self.lstm.forward(xs)
    return hs

  def backward(self, dhs) :
    dout = self.lstm.backward(dhs)
    dout = self.embed.backward(dout)
    return dout

### Decoder 구현


In [11]:
class AttentionDecoder :
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    V, D, H = vocab_size, wordvec_size, hidden_size
    rn = np.random.randn

    embed_W = (rn(V,D) / 100).astype('f')
    lstm_Wx = (rn(D,4*H) / np.sqrt(D)).astype('f')
    lstm_Wh = (rn(H,4*H) / np.sqrt(H)).astype('f')
    lstm_b = np.zeros(4*H).astype('f')
    affine_W = (rn(2*H, V)).astype('f')
    affine_b = np.zeros(V).astype('f')

    self.embed = TimeEmbedding(embed_W)
    self.lstm = TimeLSTM(lstm_Wx, lstm_Wh, lstm_b, stateful=True)
    self.attention = TimeAttention()
    self.affine = TimeAffine(affine_W, affine_b)
    layers = [self.embed, self.lstm, self.attention, self.affine]

    self.params, self.grads = [], []
    for layer in layers :
      self.params += layer.params
      self.grads += layer.grads

  def forward(self, xs, enc_hs) :
    h = enc_hs[:, -1]
    self.lstm.set_state(h)

    out = self.embed.forward(xs)
    dec_hs = self.lstm.forward(out)
    c = self.attention.forward(enc_hs, dec_hs)
    out = np.concatenate((c, dec_hs), axis=2)
    score = self.affine.forward(out)

    return score

  def backward(self, dscore):
    dout = self.affine.backward(dscore)
    N, T, H2 = dout.shape
    H = H2 // 2

    dc, ddec_hs0 = dout[:,:,:H], dout[:,:,H:]
    denc_hs, ddec_hs1 = self.attention.backward(dc)
    ddec_hs = ddec_hs0 + ddec_hs1
    dout = self.lstm.backward(ddec_hs)
    dh = self.lstm.dh
    denc_hs[:, -1] += dh
    self.embed.backward(dout)

    return denc_hs

  def generate(self, enc_hs, start_id, sample_size):
    sampled = []
    sample_id = start_id
    h = enc_hs[:, -1]
    self.lstm.set_state(h)

    for _ in range(sample_size):
        x = np.array([sample_id]).reshape((1, 1))

        out = self.embed.forward(x)
        dec_hs = self.lstm.forward(out)
        c = self.attention.forward(enc_hs, dec_hs)
        out = np.concatenate((c, dec_hs), axis=2)
        score = self.affine.forward(out)

        sample_id = np.argmax(score.flatten())
        sampled.append(sample_id)

    return sampled

In [12]:
from ch07.seq2seq import Encoder, Seq2seq

class AttentionSeq2seq(Seq2seq) :
  def __init__(self, vocab_size, wordvec_size, hidden_size) :
    args = vocab_size, wordvec_size, hidden_size
    self.encoder = AttentionEncoder(*args)
    self.decoder = AttentionDecoder(*args)
    self.softmax = TimeSoftmaxWithLoss()

    self.params = self.encoder.params + self.decoder.params
    self.grads = self.encoder.grads + self.decoder.grads

### 날짜 형식 변환 문제

In [15]:
import numpy as np
from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from ch08.attention_seq2seq import AttentionSeq2seq
from ch07.seq2seq import Seq2seq

%cd /content/drive/MyDrive/deep_learning2/ch07/
from ch07.peeky_seq2seq import PeekySeq2seq
%cd /content/drive/MyDrive/deep_learning2/

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
char_to_id, id_to_char = sequence.get_vocab()

# 입력 문장 반전
x_train, x_test = x_train[:, ::-1], x_test[:, ::-1]

# 하이퍼파라미터 설정
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 128
max_epoch = 10
max_grad = 5.0

model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

acc_list = []
for epoch in range(max_epoch) :
  trainer.fit(x_train, t_train, max_epoch=1, batch_size=batch_size, max_grad=max_grad)
  correct_num = 0

  for i in range(len(x_test)) :
    question, correct = x_test[[i]], t_test[[i]]
    verbose = i < 10
    correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose, is_reverse=True)

  acc = float(correct_num) / len(x_test)
  acc_list.append(acc)
  print('val acc %.3f%%' % (acc * 100))

model.save_params()

/content/drive/MyDrive/deep_learning2/ch07
/content/drive/MyDrive/deep_learning2
| 에폭 1 |  반복 1 / 351 | 시간 0[s] | 손실 4.08
| 에폭 1 |  반복 21 / 351 | 시간 12[s] | 손실 3.09
| 에폭 1 |  반복 41 / 351 | 시간 25[s] | 손실 1.90
| 에폭 1 |  반복 61 / 351 | 시간 38[s] | 손실 1.72
| 에폭 1 |  반복 81 / 351 | 시간 51[s] | 손실 1.46
| 에폭 1 |  반복 101 / 351 | 시간 63[s] | 손실 1.19
| 에폭 1 |  반복 121 / 351 | 시간 76[s] | 손실 1.14
| 에폭 1 |  반복 141 / 351 | 시간 88[s] | 손실 1.09
| 에폭 1 |  반복 161 / 351 | 시간 100[s] | 손실 1.06
| 에폭 1 |  반복 181 / 351 | 시간 113[s] | 손실 1.04
| 에폭 1 |  반복 201 / 351 | 시간 125[s] | 손실 1.03
| 에폭 1 |  반복 221 / 351 | 시간 138[s] | 손실 1.02
| 에폭 1 |  반복 241 / 351 | 시간 150[s] | 손실 1.02
| 에폭 1 |  반복 261 / 351 | 시간 163[s] | 손실 1.01
| 에폭 1 |  반복 281 / 351 | 시간 175[s] | 손실 1.00
| 에폭 1 |  반복 301 / 351 | 시간 188[s] | 손실 1.00
| 에폭 1 |  반복 321 / 351 | 시간 200[s] | 손실 1.00
| 에폭 1 |  반복 341 / 351 | 시간 213[s] | 손실 1.00
Q 10/15/94                     
T 1994-10-15
☒ 1978-08-11
---
Q thursday, november 13, 2008  
T 2008-11-13
☒ 1978-08-11
---
